In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import h5py, os,pathlib, torch, torchani, copy
from torchani.models import ANI1x 
from pathlib import Path

from trainer import *
from sampler import *

you are using /home/shuhao/softwares/miniconda3/envs/autoani/lib/python3.8/site-packages/torchani/


In [3]:
pooldir = Path('/home/shuhao/AMD_demo/pool')

MAX_AL_EPOCH = 2

gpu_list = ["cuda:1"]

model = torchani.models.ANI1x().to(gpu_list[0])
#we will train on relative energy, so disable the energy_shifter in the model
model.energy_shifter.self_energies = torch.tensor([ 0.0, 0.0, 0.0, 0.0]).double().to(gpu_list[0])

for epoch in range(MAX_AL_EPOCH):
    energy_shifter = torchani.utils.EnergyShifter(None)
    training, validation = torchani.data.load(pooldir).subtract_self_energies(energy_shifter).species_to_indices().split(0.8, None)
    training = training.collate(batch_size).cache()
    validation = validation.collate(batch_size).cache()
    model = train(model, training, validation)
    
    new_dataset = pooldir/('ext%d.h5'%(epoch+1))
    #I do not have multiple GPU on this test machine for sampling, so just use the same one
    model_pool = [model]
    for h5file in pooldir.glob('*.h5'):
        _ = dataset_sampler(h5file, model_pool, save_to=str(new_dataset))
    print("%d al epoch done!"%(epoch+1))
    

training starting from epoch 1
RMSE: 92.88716841242558 at epoch 2


epoch 1: 100%|██████████| 47/47 [00:03<00:00, 13.23it/s]

best RMSE at this time is:92.887


extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext1.h5
1 al epoch done!
training starting from epoch 1
RMSE: 76.41433596536204 at epoch 2


epoch 1: 100%|██████████| 47/47 [00:03<00:00, 13.26it/s]

best RMSE at this time is:76.414


extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext2.h5
extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext2.h5
2 al epoch done!
